In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "5"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch

In [2]:
from src.playg_v2.funcs import testkernel, copykernel

INCLUDE: ['/local00/bioinf/beck/envs/xlstmdev1/lib/python3.11/site-packages/torch/include', '/local00/bioinf/beck/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/local00/bioinf/beck/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/TH', '/local00/bioinf/beck/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/THC', '/local00/bioinf/beck/envs/xlstmdev1/include']
/system/user/publicwork/beck/repos/xlstm/xlstm/models/vector_lstms/vkernels/cuda/src

/local00/bioinf/beck/envs/xlstmdev1/lib


Using /system/user/beck/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /system/user/beck/.cache/torch_extensions/py311_cu118/playg_v1/build.ninja...
Building extension module playg_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] /local00/bioinf/beck/envs/xlstmdev1/bin/x86_64-conda-linux-gnu-c++ -MMD -MF funcs.o.d -DTORCH_EXTENSION_NAME=playg_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /local00/bioinf/beck/envs/xlstmdev1/lib/python3.11/site-packages/torch/include -isystem /local00/bioinf/beck/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /local00/bioinf/beck/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/TH -isystem /local00/bioinf/beck/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/THC -isystem /local00/bioinf/beck/envs/xlstmdev1/include -isystem /local00/bioinf/beck/envs/xlstmdev1/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /system/user/publicwork/beck/repos/xlstm/xlstm/models/vector_lstms/vkernels/cuda/src/playg_v2/funcs.cc -o funcs.o 
[2/3] /local00/bioinf/beck/envs/xlstmdev1/bin/nvcc  -ccbin /local00/bioinf/beck

Loading extension module playg_v1...


In [3]:
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")

In [4]:
mat_A = torch.arange(4).reshape(2, 2).to(dtype=DTYPE, device=DEVICE)
mat_A

tensor([[0., 1.],
        [2., 3.]], device='cuda:0', dtype=torch.bfloat16)

In [5]:
out = testkernel(mat_A)
out

Test kernel!


tensor([[0., 1.],
        [2., 3.]], device='cuda:0', dtype=torch.bfloat16)

In [6]:
out = copykernel(mat_A)
out 

before kernel dispatch!
rows: 2, cols: 2
blocksxy: 1-1, threads: 32-32


tensor([[0., 1.],
        [2., 3.]], device='cuda:0', dtype=torch.bfloat16)

### pytorch version

In [7]:
from template import qkv_vlstm_kernel_pytorch, qkv_vlstm_no_dmatrix_kernel_pytorch

In [8]:
H = 6 # hidden size
S = 5 # seq len
B = 1 # batch size
NH = 2 # num heads
DH = H // NH # dim per head
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")
assert H % NH == 0

In [9]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ds = torch.rand((B, NH, S, S), device=DEVICE, dtype=DTYPE)

max_log_D, _ = torch.max(ds.view(B, NH, -1), dim=-1, keepdim=True)  # (B, NH, 1)
log_D_matrix_stabilized = ds - max_log_D.unsqueeze(-1)  # (B, NH, S, S) = (B, NH, S, S) - (B, NH, 1, 1)
D_matrix = torch.exp(log_D_matrix_stabilized)  # (B, NH, S, S)
mval = torch.exp(-max_log_D.unsqueeze(-1))

In [10]:
out = qkv_vlstm_kernel_pytorch(queries=qs, keys=ks, values=vs, D_matrix=D_matrix, mval=mval)
out

tensor([[[[-2.5625,  0.4277,  1.5859],
          [-2.6719,  0.4082,  2.4375],
          [-1.6562, -0.0201,  1.0469],
          [ 2.2812,  0.1807, -1.3750],
          [ 1.4219,  0.2266, -1.0469]],

         [[-1.3672, -0.5195,  0.0229],
          [ 0.0544,  0.8789, -0.6289],
          [ 1.2109, -0.1963, -0.1797],
          [-0.7578, -0.2031,  0.3984],
          [ 0.8203,  0.1748, -0.3867]]]], device='cuda:0',
       dtype=torch.bfloat16)

In [11]:
out = qkv_vlstm_no_dmatrix_kernel_pytorch(queries=qs, keys=ks, values=vs, mval=mval, qk_decmask_normalize=True)
out

tensor([[[[-1.7812,  0.0952,  1.2344],
          [-2.4062,  0.6016,  2.5000],
          [-1.5391,  0.0986,  0.9258],
          [ 2.6562,  0.3281, -2.0156],
          [ 2.0312,  0.0781, -1.4219]],

         [[-1.1328, -0.8398,  0.0079],
          [ 0.0226,  0.9219, -0.5938],
          [ 1.5781, -0.4102, -0.0610],
          [-0.7461, -0.2178,  0.3613],
          [ 0.6758,  0.0913, -0.4316]]]], device='cuda:0',
       dtype=torch.bfloat16)

In [12]:
out = qkv_vlstm_no_dmatrix_kernel_pytorch(queries=qs, keys=ks, values=vs, mval=mval, qk_decmask_normalize=False)
out

tensor([[[[-8.3125,  0.4434,  5.7812],
          [-1.6172,  0.4043,  1.6719],
          [-3.1406,  0.2002,  1.8906],
          [11.5000,  1.4219, -8.6875],
          [ 4.9375,  0.1885, -3.4375]],

         [[-1.6562, -1.2344,  0.0131],
          [ 0.0767,  3.2656, -2.0938],
          [ 3.8438, -1.0000, -0.1465],
          [-5.0000, -1.4531,  2.4062],
          [ 3.1875,  0.4297, -2.0312]]]], device='cuda:0',
       dtype=torch.bfloat16)

In [13]:
qs, qs.shape

(tensor([[[[-0.9258, -0.4258, -2.6406],
           [ 0.1455, -0.1211, -0.5781],
           [-0.6211, -0.3281, -1.0781],
           [-0.3633, -1.6719,  2.2656],
           [ 0.3125, -0.1846,  1.2891]],
 
          [[ 1.1797, -0.1270,  1.2188],
           [ 1.4375,  1.0625, -0.4941],
           [-1.4219, -0.7227, -1.2969],
           [ 0.0698, -0.0074,  1.8984],
           [ 0.6875, -0.0781, -0.8359]]]], device='cuda:0',
        dtype=torch.bfloat16),
 torch.Size([1, 2, 5, 3]))

In [14]:
out = qkv_vlstm_no_dmatrix_kernel_pytorch(queries=qs, keys=qs, values=qs, qk_decmask_normalize=False)
out

tensor([[[[ -8.7500,   4.1562, -41.7500],
          [ -1.4297,   1.2188,  -8.1875],
          [ -4.1250,   0.9453, -17.2500],
          [  3.4844, -11.2500,  37.7500],
          [  3.6250,  -3.4062,  20.8750]],

         [[  9.3125,   2.9375,  11.8750],
          [ 10.0000,   5.0000,  -0.4219],
          [-12.9375,  -4.9375, -13.2500],
          [  4.4375,   0.7461,  14.8125],
          [  2.1406,   1.2266,  -5.0000]]]], device='cuda:0',
       dtype=torch.bfloat16)

In [21]:
warp_size = 32
am = torch.ones((1*1*warp_size, 1*1*warp_size), device=DEVICE, dtype=DTYPE)
bm = 2 * torch.ones((1*1*warp_size, 1*1*warp_size), device=DEVICE, dtype=DTYPE)
cm = am @ bm
cm, cm.shape

(tensor([[64., 64., 64.,  ..., 64., 64., 64.],
         [64., 64., 64.,  ..., 64., 64., 64.],
         [64., 64., 64.,  ..., 64., 64., 64.],
         ...,
         [64., 64., 64.,  ..., 64., 64., 64.],
         [64., 64., 64.,  ..., 64., 64., 64.],
         [64., 64., 64.,  ..., 64., 64., 64.]], device='cuda:0',
        dtype=torch.bfloat16),
 torch.Size([32, 32]))